In [2]:
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_ollama.llms import OllamaLLM


In [ ]:
# 1. Función para cargar datos desde un archivo PDF
def extract_text_from_pdf(pdf_path):
    """
    Extrae texto de todas las páginas de un archivo PDF.
    
    Args:
        pdf_path (str): Ruta al archivo PDF.
        
    Returns:
        str: Texto combinado de todas las páginas del PDF.
    """
    try:
        with open(pdf_path, "rb") as file:
            reader = PyPDF2.PdfReader(file)
            text = ""
            for page in reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        raise Exception(f"Error al leer el archivo PDF: {e}")
    
    if not text.strip():
        raise Exception("No se encontró texto significativo en el archivo PDF.")
    
    return text.strip()


# Ruta al archivo PDF
pdf_path = "./resources/archivo.pdf"  # Ruta al archivo PDF

# Extraer texto del PDF
pdf_text = extract_text_from_pdf(pdf_path)
print(f"Texto extraído del PDF (primeros 500 caracteres): {pdf_text[:500]}...\n")  # Muestra un resumen del contenido extraído

In [4]:
# 2. Dividir el texto en fragmentos (chunks) con LangChain
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_text(pdf_text)
print(f"Se generaron {len(chunks)} fragmentos de texto.\n")

Se generaron 2780 fragmentos de texto.



In [5]:
# 3. Crear embeddings usando HuggingFace
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Crear la base de datos vectorial con Chroma
vectorstore = Chroma.from_texts(
    texts=chunks,
    embedding=embeddings,
    collection_name="pdf_facts",
    persist_directory="chroma_db"  # Carpeta para guardar la base de datos
)

# Persistir la base de datos en disco
vectorstore.persist()
print(f"Base de datos vectorial creada y guardada con {len(chunks)} documentos.\n")


/tmp/ipykernel_47004/3721682011.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Base de datos vectorial creada y guardada con 2780 documentos.



/tmp/ipykernel_47004/3721682011.py:13: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [6]:
# 4. Configurar el modelo LLM con Ollama
llm = OllamaLLM(model="llama3.2", format="json", server_url="http://localhost:11434")

In [7]:
# 5. Crear el sistema RAG con Chroma
retriever = vectorstore.as_retriever()
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

In [14]:
# 6. Hacer preguntas al sistema
query = "¿Cómo está la era de la información?"

In [15]:
respuesta = qa_chain.invoke({"query": query})
print(f"Respuesta a la consulta: {respuesta}")

Respuesta a la consulta: {'query': '¿Cómo está la era de la información?', 'result': '{\n  "answer": "No sé", \n  "rasondo": "La pregunta es un párrafo que presenta una perspectiva crítica sobre la Era de la Información, destacando cómo esta revolución tecnológica está socavando las instituciones estatales y liberando oportunidades para los expertos. También menciona la posibilidad de nuevas formas de gobierno emergiendo como resultado de esta transición.", \n  "contexto": "El texto proporciona una visión optimista pero crítica sobre el impacto de la Era de la Información en la sociedad, destacando su potencial para cambiar las dinámicas políticas y económicas."}'}
